In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import gc
import nltk
nltk.download('stopwords')
import math
import pickle
import os
from smart_open import open
from nltk.corpus import stopwords
import sklearn
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors

import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras import layers


from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
dev_location = "dev_data"
test_location = "test_data"
train_location = "train_data"

devData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
testData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
trainData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]


dataset = np.concatenate((devData, np.concatenate((testData, trainData))))

gc.collect()
max_num_words = 17
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/MCA Dataset/Copy of GoogleNews-vectors-negative300.bin.gz', binary=True)
stop_words = set(stopwords.words('english'))


In [ ]:
def getTextData(patientID, location):
  fileName = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/" + str(int(patientID)) + "_TRANSCRIPT.csv"
  file = np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))



  # Remove All Utterences By Ellie:
  # for i in range(len(file)):
  #   if(file[i][2] != 'Participant'):
  #     np.delete(file, i)
  #     i-=1

  # Remove Speaker Columnn
  file = np.delete(file, 2, 1)
  # print(file[0],file[1])

  # Convert Text Into Word Vectors:
  w2vs = np.zeros((1, max_num_words*300))
  for i in range(len(file)):
    sentence = file[i][2]
    w2v = returnWordToVec(sentence)
    w2vs = np.concatenate((w2vs, w2v), axis = 0)
  w2vs = np.delete(w2vs, 0, 0)

  # Delete Sentences and Replace With W2Vs
  file = np.delete(file, 2, 1)
  # print(file)
  # print(file)
  file = np.concatenate((file,w2vs ), axis = 1)
  # print(file)
  return file

def remove_StopWords(sentence):
    filtered_sentence = []
    for w in sentence:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

def returnWordToVec(sentence):
  global max_num_words, stop_words, model
  sentence = str(sentence).split(" ")
  sentence = remove_StopWords(sentence)
  index_word = 0
  wordMatrix = np.zeros(max_num_words*300)
  for j in range(min(max_num_words, len(sentence))):
    try:
      word = sentence[j]
      if(word[0] == '<'):
        if(word.find('>')!=-1):
          word = word[1:-1]
        else:
          word = word[1:]
      else:
        if(word.find('>')!=-1):
          word = word[0:-1]
      ss = np.array(model[word])
      wordMatrix[index_word*300:(index_word+1)*300] = ss
      index_word+=1
    except Exception as e:
      continue
  wordMatrix = np.array(wordMatrix.reshape(1,-1))
  return wordMatrix




In [ ]:

text_test =[]
Ytest = []

In [ ]:

text= getData(datapoint[0], test_location)

text_test.append(text)
Ytest.append(datapoint[1])

In [ ]:
import sklearn
from sklearn import preprocessing


def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train==0]
  X_train_1 = X_train[Y_train==1]

  Y_train_1 = Y_train[Y_train==1]
  # print(Y_train_1.shape)
  # print(X_train_1.shape)
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train











# audio_test = np.nan_to_num(audio_test)
text_test = np.nan_to_num(text_test)


for i in range(text_test.shape[0]):
#   audio_test[i] = sklearn.preprocessing.normalize(audio_test[i])
#   # video_test[i] = sklearn.preprocessing.normalize(video_test[i])
  text_test[i] = sklearn.preprocessing.normalize(text_test[i])






(200, 250, 5100)
(200,)


In [ ]:
def Thresholding(Y_pred, threshold):
  Y_pred2 = []
  print("Y_pred: ", Y_pred.shape)
  for i in range(len(Y_pred)):
    if(Y_pred[i] < threshold):
      Y_pred2.append(0)
    else:
      Y_pred2.append(1)

  return np.array(Y_pred2)

gc.collect()



0

In [ ]:
#Text
class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)

    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")

    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")



#LSTM FOR TEXT
# first input model
class text:
  input3 = Input(shape = (250,5100))
  dense4 = Dense(1000)(input3)
  dense5 = Dense(500)(dense4)
  dense6 = Dense(250)(dense5)
  # interpretation model
  lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)(dense6)
  output = Dense(1, activation='sigmoid')(lstm)
  model = Model(inputs=input3, outputs=output)
  # summarize layers
  # print(model.summary())
  # # plot graph
  # print(plot_model(model, to_file='multiple_inputs.png'))
  optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
  def run_model(self):
    self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
    return self.model





In [ ]:
# print("-------------------------------------- TEXT (W/o GATING) SENTENCE LEVEL-------------------------------------------------")
# model1 = text()
# model = model1.run_model()

# model.fit(text_train,Ytrain, epochs=50,validation_split = 0.2, callbacks = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='min',
#     baseline=None, restore_best_weights=True), batch_size = 137)




# from sklearn.metrics import classification_report
pred = model.predict(text_test)

# # print(classification_report(Ytest,Thresholding(pred,0.5)))
# # print(classification_report(Ytest,Thresholding(pred,0.6)))
# # print(classification_report(Ytest,Thresholding(pred,0.4)))
# # print(classification_report(Ytest,Thresholding(pred,0.3)))
# # print(classification_report(Ytest,Thresholding(pred,0.7)))

-------------------------------------- TEXT (W/o GATING) SENTENCE LEVEL-------------------------------------------------
Epoch 1/50
2/2 [==============================] - 16s 4s/step - loss: 0.6933 - val_loss: 0.6912
Epoch 2/50
2/2 [==============================] - 4s 2s/step - loss: 0.6890 - val_loss: 0.6885
Epoch 3/50
2/2 [==============================] - 3s 2s/step - loss: 0.6860 - val_loss: 0.6860
Epoch 4/50
2/2 [==============================] - 3s 2s/step - loss: 0.6825 - val_loss: 0.6838
Epoch 5/50
2/2 [==============================] - 3s 2s/step - loss: 0.6787 - val_loss: 0.6815
Epoch 6/50
2/2 [==============================] - 5s 2s/step - loss: 0.6752 - val_loss: 0.6791
Epoch 7/50
2/2 [==============================] - 3s 2s/step - loss: 0.6702 - val_loss: 0.6770
Epoch 8/50
2/2 [==============================] - 3s 1s/step - loss: 0.6662 - val_loss: 0.6753
Epoch 9/50
2/2 [==============================] - 4s 3s/step - loss: 0.6620 - val_loss: 0.6742
Epoch 10/50
2/2 [======

In [ ]:
# # Save the entire model to a HDF5 file.
# model.save('LSTM_With_Gating_Sentence_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
